In [ ]:
import pandas as pd
import json
import requests
import time
import threading
import pyodbc
from concurrent.futures import ThreadPoolExecutor
import ast
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import os
from datetime import datetime, date
import re
'''
ask token
'''    
url = "https://login-p10.xiaoshouyi.com/auc/oauth2/token"
payload = {
    "grant_type": "password",
    "client_id": "351fb11916fc85504ba5f6ec3b96988b",
    "client_secret": "8f9355878504005ba782ca049d013d4e",
    "redirect_uri": "https://crm-p10.xiaoshouyi.com",
    "username": "11020964@twkd.com",
    "password": "Kd0627ubGHagpQ"
}

response = requests.post(url, data=payload)
content = response.json()
ac_token = content["access_token"]


In [ ]:

date_0 = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
date_0_scrm = int(date_0.timestamp() * 1000)

date_1 = datetime.now() + timedelta(days=1)
date_1_scrm = int(date_1.timestamp() * 1000)

date_3 = datetime.now() - timedelta(days=3)
date_3_scrm = int(date_3.timestamp() * 1000)

date_2m = datetime.now() - timedelta(days=2*30)
date_2m_scrm = int(date_2m.timestamp() * 1000)

'''
電訪人員
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
call_center = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''SELECT customItem1__c.name as name, customItem1__c as EmplID, customItem10__c as call_number, customItem5__c as region,
                          dimDepart.departName
                            from customEntity42__c
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    call_center = pd.concat([call_center, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
call_center['region'] = call_center['region'].str.get(0)
call_center = call_center[call_center['region']!='台灣']
call_center = call_center[~call_center['name'].str.contains('*', regex=False)]
call_center = call_center[(call_center['call_number'] != '') & (call_center['call_number'] != '0')]
#call_center = call_center[call_center['name']=='Floscelia']
#call_center = call_center[call_center['region'].isin(['菲律賓', '印尼'])]
#call_center = call_center[call_center['name'].isin(['郭嘉軒(ISSAC)', '趙偉成（Eric）', '林耀敏'])]

region_to_abbr = {
    '印尼': 'ID',
    '美國': 'US',
    '香港': 'HK',
    '馬來西亞': 'MY',
    '菲律賓': 'PH',
    '越南': 'VN',
    '新加坡': 'SG',
    '泰國': 'TH',
    '印度': 'IN',
    '韓國': 'KR',
    '日本': 'JP'
}
call_center['abbr'] = call_center['region'].apply(lambda x: region_to_abbr.get(x, ''))


def extract_phone2(text):
    if "Phone" in text:
        # 找到 Phone 出現的位置
        phone_index = text.find("Phone")
        # 擷取從 Phone 開始的子字串
        text = text[phone_index:]    
        # 初始化變量
        matches = []
        i = 0       
        # 使用 while 循環來遍歷字串
        while i < len(text):
            # 檢查是否遇到 "Address" 或換行符號
            if text[i:i+7] == "Address" or text[i] == "\n" or text[i] == "/":
                break
            # 使用正則表達式來查找數字
            digit_match = re.match(r'\d+', text[i:])
            if digit_match:
                matches.append(digit_match.group())
                i += digit_match.end()  # 移動索引到匹配的數字後面
            else:
                i += 1  # 如果沒有匹配到數字，則移動到下一個字符
        
        return ' '.join(matches) if matches else None  # 如果找到數字則連接成字串返回
    return None

def extract_phone3(text):
    if "N/" in text:
        # 找到 Phone 出現的位置
        phone_index = text.find("N/")
        # 擷取從 Phone 開始的子字串
        text = text[phone_index:]
        
        # 初始化變量
        matches = []
        i = 0       
        # 使用 while 循環來遍歷字串
        while i < len(text):
            # 檢查是否遇到 "Address" 或換行符號
            if text[i:i+7] == "Address" or text[i] == "\n":
                break
            # 使用正則表達式來查找數字
            digit_match = re.match(r'\d+', text[i:])
            if digit_match:
                matches.append(digit_match.group())
                i += digit_match.end()  # 移動索引到匹配的數字後面
            else:
                i += 1  # 如果沒有匹配到數字，則移動到下一個字符
        
        return ' '.join(matches) if matches else None  # 如果找到數字則連接成字串返回
    return None
def extract_address(text):
    match = re.search(r'Address:\s*(.*?)\s*Company type:', text, re.S)
    if match:
        return match.group(1).strip()
    return None


In [ ]:
'''
主營城市表
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
city_yes = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
                 select name, customItem10__c as region, regionGroupName__c.name as region2
                 from region__c
                 where customItem9__c = 1
                 ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    city_yes = pd.concat([city_yes, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
city_yes['region'] = city_yes['region'].str.get(0)

id_city = city_yes[city_yes['region']=='印尼'] 
th_city = city_yes[city_yes['region']=='泰國'] 
ph_city = city_yes[city_yes['region']=='菲律賓'] 
in_city = city_yes[city_yes['region']=='印度'] 
vn_city = city_yes[city_yes['region']=='越南'] 

In [ ]:

'''
新客電訪交辦(抓有紀錄的新客, 再繼續打)
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
record_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
                 select name, dimDepart.departName as DataRegion, customItem94__c, customItem2__c, dimDepart,
                        customItem6__c, customItem85__c,nation__c as 國家,
                        customItem91__c, zipCode__c, customItem110__c,
                        customItem88__c, ownerId, entityType, customItem120__c as 第一語言, customItem121__c as 第二語言,
                        customItem7__c as hope_date,createdAt as 創建日期, customItem74__c as reason,
                        customItem38__c as 電訪類型, customItem39__c as 是否邀約K大
                 from customEntity47__c 
                 where entityType = '3126024717160324' and createdAt >='1724025600000'
                 ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    record_df = pd.concat([record_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
record_df['電訪類型'] = record_df['電訪類型'].str.get(0)
record_df['是否邀約K大'] = record_df['是否邀約K大'].str.get(0)
record_df['hope_date'] = pd.to_numeric(record_df['hope_date'], errors='coerce') 
record_df['hope_date'] = record_df['hope_date'].astype(float)
record_df['hope_date'] = record_df['hope_date'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
record_df['hope_date'] = record_df['hope_date'].dt.tz_convert('Asia/Taipei')
record_df['hope_date'] = record_df['hope_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
record_df['創建日期'] = pd.to_numeric(record_df['創建日期'], errors='coerce') 
record_df['創建日期'] = record_df['創建日期'].astype(float)
record_df['創建日期'] = record_df['創建日期'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
record_df['創建日期'] = record_df['創建日期'].dt.tz_convert('Asia/Taipei')
record_df['創建日期'] = record_df['創建日期'].dt.strftime('%Y-%m-%d %H:%M:%S')
record_df = record_df[record_df['創建日期'] >= '2024-08-19 00:00:00']
record_df = record_df.sort_values(by='hope_date', ascending=True)
record_df['國家'] = record_df['國家'].str.get(0)
record_df['reason'] = record_df['reason'].str.get(0)
record_df['第一語言'] = record_df['第一語言'].str.get(0)
record_df['第二語言'] = record_df['第二語言'].str.get(0)
'''
get description
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/xobjects/customEntity47__c/description"
headers = {
    "Authorization": f"Bearer {ac_token}"
}

response2 = requests.get(url_2, headers = headers)
descript_json = json.loads(response2.text)
descript_data = descript_json["data"]["fields"]
description_df = pd.json_normalize(descript_data)

countryid = description_df.loc[description_df['apiKey'] == 'nation__c', 'selectitem'].to_frame()
countryid['selectitem'] = countryid['selectitem'].apply(lambda x: [{'label': d['label'], 'apiKey': d['apiKey']} for d in x])
countryid = countryid['selectitem'].explode().apply(pd.Series)[['label', 'apiKey']]
countryid.rename(columns={'apiKey': 'nation__c'}, inplace=True)
record_df = pd.merge(record_df, countryid, left_on = '國家', right_on = 'label', how = 'left')

reason = description_df.loc[description_df['apiKey'] == 'customItem74__c', 'selectitem'].to_frame()
reason['selectitem'] = reason['selectitem'].apply(lambda x: [{'label': d['label'], 'apiKey': d['apiKey']} for d in x])
reason = reason['selectitem'].explode().apply(pd.Series)[['label', 'apiKey']]
reason.rename(columns={'apiKey': 'customItem74__c'}, inplace=True)
record_df = pd.merge(record_df, reason, left_on = 'reason', right_on = 'label', how = 'left')

language1 = description_df.loc[description_df['apiKey'] == 'customItem120__c', 'selectitem'].to_frame()
language1['selectitem'] = language1['selectitem'].apply(lambda x: [{'label': d['label'], 'apiKey': d['apiKey']} for d in x])
language1 = language1['selectitem'].explode().apply(pd.Series)[['label', 'apiKey']]
language1.rename(columns={'apiKey': 'customItem120__c'}, inplace=True)
record_df = pd.merge(record_df, language1, left_on = '第一語言', right_on = 'label', how = 'left')

language2 = description_df.loc[description_df['apiKey'] == 'customItem121__c', 'selectitem'].to_frame()
language2['selectitem'] = language2['selectitem'].apply(lambda x: [{'label': d['label'], 'apiKey': d['apiKey']} for d in x])
language2 = language2['selectitem'].explode().apply(pd.Series)[['label', 'apiKey']]
language2.rename(columns={'apiKey': 'customItem121__c'}, inplace=True)
record_df.drop(columns=['label_x', 'label_y', 'label'], inplace=True)
record_df = pd.merge(record_df, language2, left_on = '第二語言', right_on = 'label', how = 'left')

record_df['phone'] = record_df['customItem94__c'].apply(extract_phone2)
record_df['phone'] = record_df['phone'].str.replace(" ", "", regex=False)
record_df['phone'] = record_df.apply(
    lambda row: extract_phone3(row['customItem94__c']) if pd.isna(row['phone']) else row['phone'],
    axis=1
)
#record_df = record_df.drop_duplicates(subset=['phone'], keep='first')
record_df['Address'] = record_df['customItem94__c'].apply(extract_address)
record_df['eight'] = record_df['phone'].str[-8:]

In [ ]:
''' 
去關係聯絡人(手機號碼主要, 聯絡人電話號碼_含區碼)
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
comp_cont = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
            SELECT id,
                   contactCode__c__c AS contact_code,
                   name AS ContactID,
                   contactPhone__c__c AS MobilePhone
            FROM customEntity22__c
            WHERE customItem37__c NOT LIKE 'TW%' AND customItem37__c NOT LIKE 'KDED%'
            ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    comp_cont = pd.concat([comp_cont, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
comp_cont['MobilePhone'] = comp_cont['MobilePhone'].str.findall(r'\d+').str.join('')
comp_cont['MobilePhone'] = comp_cont['MobilePhone'].str.replace(" ", "")
comp_cont = comp_cont[comp_cont['MobilePhone']!='']
comp_cont['MobilePhone'] = comp_cont['MobilePhone'].str[-8:]
ok_df = record_df[~record_df['eight'].isin(comp_cont['MobilePhone'])]

'''
去公司電話(公司電話_含區碼)
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
acc_phone = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
            SELECT id,
                   dimDepart.departName,
                   Phone__c,
                   customItem309__c as Phone2,
                   address
            FROM account
            WHERE dimDepart.departName NOT LIKE 'TW%' AND dimDepart.departName NOT LIKE 'KDED%'
            ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    acc_phone = pd.concat([acc_phone, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
acc_phone['Phone__c'] = acc_phone['Phone__c'].str.findall(r'\d+').str.join('')
acc_phone['Phone2'] = acc_phone['Phone2'].str.findall(r'\d+').str.join('')
acc_phone['Phone__c'] = acc_phone['Phone__c'].str[-8:]
acc_phone['Phone2'] = acc_phone['Phone2'].str[-8:]
ok_df = ok_df[~ok_df['eight'].isin(acc_phone['Phone__c'])]
ok_df = ok_df[~ok_df['eight'].isin(acc_phone['Phone2'])]
ok_df = ok_df[~ok_df['Address'].isin(acc_phone['address'])]
old_df = ok_df.copy()

'''
去掉勿電訪名單
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
no_call = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
                 select name, customItem94__c, customItem78__c as no_bother
                 from customEntity47__c 
                 where entityType = '3126024717160324' 
                 ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    no_call = pd.concat([no_call, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
no_call['no_bother'] = no_call['no_bother'].astype(str)
no_call = no_call[no_call['no_bother'].str.contains("電訪")]
old_df = old_df[~old_df['customItem94__c'].isin(no_call['customItem94__c'])]


'''
去掉今天未來K大預約 and customItem2__c >= '{date_0_scrm}'
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
no_com = pd.DataFrame()
 
while True:
    data = {
        "xoql": f'''
                 select id, customItem2__c 預約K大日期, customItem22__c 系統未建
                 from customEntity23__c
                 where entityType = '3095935350757778' and customItem2__c >= '{date_0_scrm}'
                 ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    no_com = pd.concat([no_com, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

if not no_com.empty:
    no_com['phone'] = no_com['系統未建'].apply(extract_phone2)
    no_com['phone'] = no_com['phone'].str.replace(" ", "", regex=False)
    no_com['phone'] = no_com.apply(
        lambda row: extract_phone3(row['系統未建']) if pd.isna(row['phone']) else row['phone'],
        axis=1
    )
    no_com['eight'] = no_com['phone'].str[-8:]
    no_com = no_com[~no_com['eight'].isna()]
    old_df = old_df[~old_df['eight'].isin(no_com['eight'])]


In [ ]:
'''
抽出電話, 電訪類型, 邀約K大, 並去重, 只抓最新一筆紀錄
'''
old_df = old_df.sort_values(by='創建日期', ascending=False)
no_again = old_df[
    (old_df['電訪類型'].isin(['Non-Target', '號碼無效'])) ]
old_df = old_df[~old_df['phone'].isin(no_again['phone'])]
old_df = old_df.drop_duplicates(subset='phone', keep='first')

old_df['customItem2__c'] = 'New Client Sales Call'

def process_data_region(region):
    if region.startswith('KDED'):
        return region
    else:
        return region[:2]
region_to_abbr = {
    'Indonesia Branch': 'ID',
    'United States Branch': 'US',
    '香港事業部': 'HK',
    'Malaysia Branch': 'MY',
    'Philippines Branch': 'PH',
    'Vietnam Branch': 'VN',
    'Singapore Branch': 'SG',
    'Thailand Branch': 'TH',
    'India Branch': 'IN',
    'CAN': 'CAN'
}
old_df['abbr'] = old_df['DataRegion'].apply(lambda x: region_to_abbr.get(x, ''))
old_df['abbr'] = old_df.apply(
    lambda row: process_data_region(row['DataRegion']) if not row['abbr'] else row['abbr'], axis=1
)

'''
抓出所有主營城市
'''
india_df = old_df[old_df['abbr']=='IN']
name_pattern = '|'.join(in_city['name'].values)
region2_pattern = '|'.join(in_city['region2'].values)
pattern_in = f"{name_pattern}|{region2_pattern}"
india_df = india_df[india_df['Address'].str.contains(pattern_in, flags=re.IGNORECASE, na=False)]

ph_df = old_df[old_df['abbr']=='PH']
name_pattern = '|'.join(ph_city['name'].values)
region2_pattern = '|'.join(ph_city['region2'].values)
pattern_ph = f"{name_pattern}|{region2_pattern}"
ph_df = ph_df[ph_df['Address'].str.contains(pattern_ph, flags=re.IGNORECASE, na=False)]

indo_df = old_df[old_df['abbr']=='ID']
name_pattern = '|'.join(id_city['name'].values)
region2_pattern = '|'.join(id_city['region2'].values)
pattern_id = f"{name_pattern}|{region2_pattern}"
indo_df = indo_df[indo_df['Address'].str.contains(pattern_id, flags=re.IGNORECASE, na=False)]

vn_df = old_df[old_df['abbr']=='VN']
name_pattern = '|'.join(vn_city['name'].values)
region2_pattern = '|'.join(vn_city['region2'].values)
pattern_vn = f"{name_pattern}|{region2_pattern}"
vn_df = vn_df[vn_df['Address'].str.contains(pattern_vn, flags=re.IGNORECASE, na=False)]

th_df = old_df[old_df['abbr']=='TH']
name_pattern = '|'.join(th_city['name'].values)
region2_pattern = '|'.join(th_city['region2'].values)
pattern_th = f"{name_pattern}|{region2_pattern}"
th_df = th_df[th_df['Address'].str.contains(pattern_th, flags=re.IGNORECASE, na=False)]

left = old_df[old_df['abbr'].isin(['SG', 'HK'])]

old_df = pd.concat([india_df, ph_df, indo_df, vn_df, th_df, left])
# =============================================================================
# pattern = '|'.join(city_yes['name'].values)
# five_country = old_df[~old_df['abbr'].isin(['SG', 'HK'])]
# five_country = five_country[five_country['Address'].str.contains(pattern, flags=re.IGNORECASE, na=False)]
# left = old_df[old_df['abbr'].isin(['SG', 'HK'])]
# old_df = pd.concat([five_country, left])
# =============================================================================

'''
不和今天以及未來重複
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
today_SCRM = pd.DataFrame()
 
while True:
    data = {
        "xoql": f'''SELECT 
                            id,name, customItem94__c, customItem110__c
                            from customEntity47__c
                            where entityType = '3126024717160324' 
                            and customItem7__c >= {date_0_scrm} and customItem1__c is not null
                            
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    today_SCRM = pd.concat([today_SCRM, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
today_SCRM['customItem94__c'] = today_SCRM['customItem94__c'].str.replace('=+', '', regex=False)
today_SCRM['customItem94__c'] = today_SCRM['customItem94__c'].str.replace('=', '', regex=False)
today_SCRM['phone'] = today_SCRM['customItem94__c'].apply(extract_phone2)
today_SCRM['phone'] = today_SCRM['phone'].str.replace(" ", "", regex=False)
today_SCRM['eight'] = today_SCRM['phone'].str[-8:]
old_df = old_df[~old_df['eight'].isin(today_SCRM['eight'])]


'''
3天內不打到一樣的
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
three_SCRM = pd.DataFrame()
 
while True:
    data = {
        "xoql": f'''SELECT 
                            id,name, customItem94__c, customItem110__c
                            from customEntity47__c
                            where entityType = '3126024717160324' 
                            and customItem7__c >= {date_3_scrm} and customItem1__c is not null
                            
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    three_SCRM = pd.concat([three_SCRM, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
three_SCRM['customItem94__c'] = three_SCRM['customItem94__c'].str.replace('=+', '', regex=False)
three_SCRM['customItem94__c'] = three_SCRM['customItem94__c'].str.replace('=', '', regex=False)
three_SCRM['phone'] = three_SCRM['customItem94__c'].apply(extract_phone2)
three_SCRM['phone'] = three_SCRM['phone'].str.replace(" ", "", regex=False)
three_SCRM['phone'] = three_SCRM.apply(
    lambda row: extract_phone3(row['customItem94__c']) if pd.isna(row['phone']) else row['phone'],
    axis=1
)
three_SCRM['phone'] = three_SCRM.apply(lambda row: extract_phone2(row['customItem94__c']) if row['phone'] == '' else row['phone'], axis=1)
three_SCRM['eight'] = three_SCRM['phone'].str[-8:]
old_df = old_df[~old_df['eight'].isin(three_SCRM['eight'])]


'''
未接3去掉(建檔日期 兩個月內 計算)
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
no_three_call = pd.DataFrame()
 
while True:
    data = {
        "xoql": f'''SELECT 
                            id,name, customItem94__c, customItem38__c as 電訪類型
                            from customEntity47__c
                            where entityType = '3126024717160324' and customItem36__c >= {date_2m_scrm}
                            
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    no_three_call = pd.concat([no_three_call, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
no_three_call['電訪類型'] = no_three_call['電訪類型'].str.get(0)
no_three_call = no_three_call[no_three_call['電訪類型']=='未接']
no_three_call['customItem94__c'] = no_three_call['customItem94__c'].str.replace('=+', '', regex=False)
no_three_call['customItem94__c'] = no_three_call['customItem94__c'].str.replace('=', '', regex=False)
no_three_call['phone'] = no_three_call['customItem94__c'].apply(extract_phone2)
no_three_call['phone'] =no_three_call['phone'].str.replace(" ", "", regex=False)
no_three_call['phone'] = no_three_call.apply(
    lambda row: extract_phone3(row['customItem94__c']) if pd.isna(row['phone']) else row['phone'],
    axis=1
)
no_three_call['phone'] = no_three_call.apply(lambda row: extract_phone2(row['customItem94__c']) if row['phone'] == '' else row['phone'], axis=1)
phone_counts = no_three_call.groupby('phone').size().reset_index(name='count')
phone_counts = phone_counts[phone_counts['count']>2]
old_df = old_df[~old_df['phone'].isin(phone_counts['phone'])]


old_df['customItem1__c']=''
old_df['Operator'] = pd.NA
old_df['EmplID'] = pd.NA  

final_all = old_df.copy()
final_all = final_all.drop_duplicates(subset='phone', keep='first')

In [ ]:

'''
去追蹤紀錄(二面陌生開發)
accountCode__c = 'GAC00149237' 二面存放追蹤紀錄用
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
tr_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''SELECT customItem145__c, customItem177__c as 電訪類型
                            from customEntity15__c
                            where entityType = '3073342820798377' and accountCode__c = 'GAC00149237'
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    tr_df = pd.concat([tr_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

tr_df['phone'] = tr_df['customItem145__c'].apply(extract_phone2)
tr_df['phone'] = tr_df['phone'].str.replace(" ", "", regex=False)
tr_df['phone'] = tr_df.apply(
    lambda row: extract_phone3(row['customItem145__c']) if pd.isna(row['phone']) else row['phone'],
    axis=1
)
tr_df['電訪類型'] = tr_df['電訪類型'].str.get(0)
no_again = tr_df[
    (tr_df['電訪類型'].isin(['Non-Target', 'Duplicate', '號碼無效'])) ]
final_all = final_all[~final_all['phone'].isin(no_again['phone'])]

#打亂
final_all = final_all.sample(frac=1).reset_index(drop=True)  


In [ ]:
'''
count total cases for new clients
'''
total_cases = final_all.groupby('abbr').size().reset_index(name='total_number')
call_center['call_number'] = call_center['call_number'].astype(int)
total_call_numbers = call_center.groupby('abbr')['call_number'].sum().reset_index(name='total_call_numbers')
call_center = call_center.merge(total_cases, left_on='abbr', right_on = 'abbr',  how='left')
call_center = call_center.merge(total_call_numbers, on='abbr',  how='left')
condition = call_center['total_number'] < call_center['total_call_numbers']
# Calculate adjusted_number based on the condition
call_center['adjusted_number'] = np.floor(call_center['total_number'] * call_center['call_number'] / call_center['total_call_numbers'])
# Where total_number <= total_call_numbers, set adjusted_number equal to call_number
call_center.loc[~condition, 'adjusted_number'] = call_center['call_number']    

employee_calls = {}  # Initialize the dictionary

for index, row in call_center.iterrows():
    employee_calls[row['name']] = int(row['adjusted_number'])  # Initialize each employee's call count
    region = row['abbr']
    call_number = int(row['adjusted_number'])
    region_data = final_all[final_all['abbr'] == region]
    remaining_calls = len(region_data)
    sum_number = sum(int(call_num) for call_num in call_center[call_center['abbr'] == region]['adjusted_number'])
    same_region_people = call_center[call_center['abbr'] == region]['name']
    count = len(same_region_people)  # Count of people in the same region, excluding the current person
    total_people = count   # Total number of people in the same region, including the current person

    if sum_number < remaining_calls:
        cases_to_assign = min(call_number, len(region_data))
        if cases_to_assign > 0:
            employee_name = row['name']  # Get the employee name
            employee_index = region_data[region_data['Operator'].isnull()].index  # Get indices of rows where Operator is not assigned
            cases_for_employee = min(cases_to_assign, len(employee_index))  # Limit cases to available rows

            # Assign Operator and EmplID to the selected rows
            final_all.loc[employee_index[:cases_for_employee], 'Operator'] = employee_name
            final_all.loc[employee_index[:cases_for_employee], 'EmplID'] = row['EmplID']

            remaining_calls -= cases_for_employee
            employee_calls[employee_name] -= cases_for_employee
    else:
        cases_to_assign = remaining_calls // total_people  # Distribute remaining calls evenly
        for employee_name in employee_calls:
            employee_index = region_data[region_data['Operator'].isnull()].index
            cases_for_employee = min(cases_to_assign, len(employee_index))
            final_all.loc[employee_index[:cases_for_employee], 'Operator'] = employee_name
            final_all.loc[employee_index[:cases_for_employee], 'EmplID'] = row['EmplID']
            remaining_calls -= cases_for_employee
            employee_calls[employee_name] -= cases_for_employee
final_all = final_all[~final_all['Operator'].isna()]
call_center_final = call_center[['name', 'abbr']]
final = pd.merge(final_all, call_center_final, left_on = 'Operator', right_on = 'name', how = 'left')
final = final[final['abbr_x']==final['abbr_y']]
final = final.drop_duplicates(subset=['phone'], keep='first')
final['customItem7__c'] = date_0_scrm

'''
最後放入需要的欄位
'''
final = final[['EmplID', 'customItem94__c', 'customItem2__c', 'customItem7__c', 'dimDepart', 'ownerId',
           'customItem6__c', 'customItem85__c', 'customItem91__c', 'zipCode__c', 'customItem110__c',
           'customItem88__c', 'nation__c', 'entityType', 'customItem74__c', 'customItem120__c',
           'customItem121__c']]
final = final.rename(columns={'EmplID': 'customItem1__c'})
final = final[~final['customItem1__c'].isna()]
final['nation__c'] = final['nation__c'].fillna('')
final = final.fillna('')
'''
ask bulk id
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/bulk/v2/job"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/json"
}
data = {
     "data": {
        "operation": "insert",####################################################################3
        "object": "customEntity47__c",
        #"execOption": ["CHECK_RULE", "CHECK_DUPLICATE"]
    }
    
}
response = requests.post(url_2, headers = headers, json = data)
test = response.json()
bulk_id = test["result"]["id"]
print(bulk_id)

'''
insert to crm if dataframe over 5000
'''
batch_size = 5000

# Calculate the number of batches
num_batches = (len(final) + batch_size - 1) // batch_size
# Initialize a list to store responses
all_responses = []
# Initialize an index to keep track of the current batch
current_batch_index = 0

# While there are more batches to process
while current_batch_index < num_batches:
    start_index = current_batch_index * batch_size
    end_index = (current_batch_index + 1) * batch_size
    current_batch_df = final.iloc[start_index:end_index]
    json_data = current_batch_df.to_dict(orient='records')
    
    url_2 = "https://api-p10.xiaoshouyi.com/rest/bulk/v2/batch"
    headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type":"application/json"
    }
    data = {
        "data": {
            "jobId": f"{bulk_id}",
            "datas": json_data
        }
    }
    response = requests.post(url_2, headers=headers, json=data)
    all_responses.append(response)
    current_batch_index += 1
for response in all_responses:
    pass